# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

In [83]:
from xml.etree import ElementTree as ET

## XML example

+ for details about tree traversal and iterators, see https://docs.python.org/2.7/library/xml.etree.elementtree.html

In [84]:
document_tree = ET.parse( './data/mondial_database_less.xml' )

In [85]:
# print names of all countries
for child in document_tree.getroot():
    print (child.find('name').text)


Albania
Greece
Macedonia
Serbia
Montenegro
Kosovo
Andorra


In [86]:
# print names of all countries and their cities
for element in document_tree.iterfind('country'):
    print ('* ' + element.find('name').text + ':'),
    capitals_string = ''
    for subelement in element.getiterator('city'):
        capitals_string += subelement.find('name').text + ', '
    print (capitals_string[:-2])

* Albania:
Tirana, Shkodër, Durrës, Vlorë, Elbasan, Korçë
* Greece:
Komotini, Kavala, Athina, Peiraias, Peristeri, Acharnes, Patra, Kozani, Kerkyra, Ioannina, Thessaloniki, Iraklio, Chania, Ermoupoli, Rhodes, Tripoli, Lamia, Chalkida, Larissa, Volos, Mytilini, Karyes
* Macedonia:
Skopje, Kumanovo
* Serbia:
Beograd, Novi Sad, Niš
* Montenegro:
Podgorica
* Kosovo:
Prishtine
* Andorra:
Andorra la Vella


****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [87]:
import pandas as pd
import numpy as np
document = ET.parse( './data/mondial_database.xml' )
root =document.getroot()
# make two lists to store infant mortality rate and country name
infant_mort = list()
country_name = list()

# loop to find country name and add to the list only if infant mortality is provided for that country
for country in root.findall('country'):
    if country.findall('infant_mortality'):
    
        country_name.append(country.find('name').text)
        
        # convert the infant mortality text to float before appending it to the list
        infant_mort.append(float(country.find('infant_mortality').text))

# create a dataframe to store infant mortality and country        
df = pd.DataFrame({'infant_mortality': infant_mort, 'country': country_name})

# sort the values by infant_mortality and show 10 records with lowest mortality rates.
df.sort_values(by = 'infant_mortality', ascending = True).head(10)


,country,infant_mortality
36,Monaco,1.81
90,Japan,2.13
109,Bermuda,2.48
34,Norway,2.48
98,Singapore,2.53
35,Sweden,2.60
8,Czech Republic,2.63
72,Hong Kong,2.73
73,Macao,3.13
39,Iceland,3.15


In [88]:
# 10 cities with the largest population

# making 4 list such that they can store values from xml file and later on use it in a dataframe.
pop = list()
city_name = list()
country_name = list()
year = list()

# loop through the country
for country in document.findall('country'):
    
#store city and cities within province
    cities = country.findall('city')
    province = country.findall('province')
    for prov in province:
        cities += prov.findall('city')
    for cit in cities:
        
# if population exist in a city then add city name, country name, population, and year into the corresponding lists.
        if cit.findall('.//population[last()]'):
            city_name.append(cit.find('name').text)
            country_name.append(country.find('name').text)
            pop.append(int(cit.find('.//population[last()]').text))
            year.append(cit.find('.//population[last()]').get('year'))
            
# make a dataframe that would have four columns and can later be sorted to get 10 cities with the largest population
df = pd.DataFrame({'country': country_name, 'city_name': city_name, 'year': year, 'population': pop})

df.sort_values(by = 'population', ascending = False).head(10)

,city_name,country,population,year
1251,Shanghai,China,22315474,2010
707,Istanbul,Turkey,13710512,2012
1421,Mumbai,India,12442373,2011
443,Moskva,Russia,11979529,2013
1250,Beijing,China,11716620,2010
2594,São Paulo,Brazil,11152344,2010
1252,Tianjin,China,11090314,2010
974,Guangzhou,China,11071424,2010
1467,Delhi,India,11034555,2011
977,Shenzhen,China,10358381,2010


In [89]:
# 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)

# make 2 list that would store name of ethnic group and population of that group
ethnicgroup = list()
ethnicpopulation = list()

# iterate through all the countries
for countries in document.findall('country'):

# find the population of the country and store it in a variable as a float value
    if countries.findall('.//population[last()]'):
        population = float(countries.find('.//population[last()]').text)

# find ethnic group of the countries if they are provided and get their population percentage. 
# Calculate the ethnic group population by multiplying the percentage and population of the country.
# Append the name of the ethnic group and population of the ethnic group in the list.
        if countries.findall('ethnicgroup'):
            ethnicgroup.append(countries.find('ethnicgroup').text)
            pop = float(countries.find('ethnicgroup').get('percentage'))
            ethnicpop = (pop * population) / (100.0)
            ethnicpopulation.append(ethnicpop)

# Make a dataframe that would store two list with name of ethnic group and population of ethnic group
# Group the dataframe by ethnic group and sum the ethnic population. Sort the values by ethnicpopulation.
df = pd.DataFrame({'ethnicgroup': ethnicgroup, 'ethnicpopulation': ethnicpopulation})
df.groupby('ethnicgroup').sum().sort_values(by = 'ethnicpopulation', ascending = False).head(10)
            
            
            

,ethnicpopulation
ethnicgroup,
Han Chinese,1.245059e+09
European,4.410033e+08
Dravidian,3.027137e+08
African,1.986050e+08
Bengali,1.467769e+08
Mestizo,1.419729e+08
Japanese,1.265342e+08
Russian,1.146462e+08
Javanese,1.134560e+08
